In [1]:
import ROOT as r
import numpy as np

Welcome to JupyROOT 6.22/06


In [2]:
file = r.TFile("generatedComponentsBOOST.root","READ")
tree = file.Get("tree") # Getting the tree 

Warning in <TFile::Init>: file generatedComponentsBOOST.root probably not closed, trying to recover
Info in <TFile::Recover>: generatedComponentsBOOST.root, recovered key TTree:tree at address 1042
Warning in <TFile::Init>: successfully recovered 1 keys


In [3]:
# Creating TLorentzVectors
eTLV = r.TLorentzVector()
nTLV = r.TLorentzVector()

In [4]:
# Initializing 4-element array to store 4-vectors
wArr = np.zeros(4)
eArr = np.zeros(4)
nArr = np.zeros(4)

In [5]:
# Taking already filled branches from the file
# Now we need to add another branch for neutrino
tree.SetBranchAddress("wBoson", wArr)
tree.SetBranchAddress("electron", eArr)

# Initializing branch for neutrino components
tree.Branch("neutrino",nArr,"neArray[4]/D")

Works in the same way as the minimizer for **stationary W**. 

I have to comment on why I chose the `initVal` (initial value for nPz).

In [6]:
%%cpp -d   

void fcn(int &npar, double *gin, double &f, double *par, int iflag){
    double pTSq = par[0];
    double pTotSq = par[1];
    double ePz = par[2];
    double nPz = par[3];

    double wMass = TMath::Sqrt(2*pTSq + 2 * TMath::Sqrt(pTotSq) * TMath::Sqrt(pTSq + pow(nPz,2)) - 2 * ePz * nPz);

    f = pow(wMass-80.3,2);
};

double minimization_func(double *wArr, double *elArr){
    double ePx = elArr[0];
    double ePy = elArr[1];
    double ePz = elArr[2];
    
    double wPz = wArr[2]; 
    
    double initVal;
    
    if(abs(ePz - wPz) > 1){
        initVal = -(ePz - wPz)+1;
    } else {
        initVal = -ePz + 0.02;  
    }
    
    double pTSqEl = pow(ePx,2) + pow(ePy,2);
    double pTotSqEl = pTSqEl + pow(ePz,2);  
    
    TMinuit *gMinuit = new TMinuit(4);
    gMinuit->SetFCN(fcn);
    gMinuit->DefineParameter(0, "pTotSqEl", pTSqEl, 1., 0., 0.);
    gMinuit->DefineParameter(1, "pTSqEl", pTotSqEl, 1., 0., 0.);
    gMinuit->DefineParameter(2, "ePz", ePz, 1., 0., 0.);
    gMinuit->DefineParameter(3, "nPz", initVal, 0.05, 0., 0.);
    gMinuit->FixParameter(0);
    gMinuit->FixParameter(1);
    gMinuit->FixParameter(2);
    gMinuit->Command("MIGRAD");

    double nPz, nPzErr;
    gMinuit->GetParameter(3, nPz, nPzErr);
    return nPz;

};

In [7]:
eArr

array([0., 0., 0., 0.])

In [8]:
n = tree.GetEntries() # Number of entries in tree (number of different events)

for i in range(n):
    tree.GetEntry(i)
    ePx = tree.GetListOfLeaves().At(1).GetValue(0)
    ePy = tree.GetListOfLeaves().At(1).GetValue(1)
    
    nArr[0] = -ePx
    nArr[1] = -ePy
    nArr[2] = r.minimization_func(wArr, eArr)
    nArr[3] = r.TMath.Sqrt(nArr[0]**2 + nArr[1]**2 + nArr[2]**2)
    
    tree.GetBranch("neutrino").Fill() # Filling just "neutrino" branch

 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 pTotSqEl     7.05613e+01  1.00000e+00     no limits
     2 pTSqEl       1.32542e+03  1.00000e+00     no limits
     3 ePz         -3.54240e+01  1.00000e+00     no limits
     4 nPz          4.44945e+01  5.00000e-02     no limits
 **********
 **    1 **FIX           1
 **********
 **********
 **    2 **FIX           2
 **********
 **********
 **    3 **FIX           3
 **********
 **********
 **    4 **MIGRAD 
 **********
 FIRST CALL TO USER FUNCTION AT NEW START POINT, WITH IFLAG=4.
 START MIGRAD MINIMIZATION.  STRATEGY  1.  CONVERGENCE WHEN EDM .LT. 1.00e-04
 FCN=0.777288 FROM MIGRAD    STATUS=INITIATE        4 CALLS           5 TOTAL
                     EDM= unknown      STRATEGY= 1      NO ERROR MATRIX       
  EXT PARAMETER               CURRENT GUESS       STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  pTotSqEl     7.05613e+01     fixed    
 

In [9]:
tree.Scan()

40

***********************************************************
*    Row   * Instance * wBoson.wB * electron. * neutrino. *
***********************************************************
*        0 *        0 *         0 * -8.213797 * 8.2137975 *
*        0 *        1 *         0 * -1.759209 * 1.7592096 *
*        0 *        2 * 8.0704536 * -35.42401 * 43.494718 *
*        0 *        3 * 80.704536 * 36.406344 * 44.298440 *
*        1 *        0 *         0 * -29.09737 * 29.097373 *
*        1 *        1 *         0 * 0.0803621 * -0.080362 *
*        1 *        2 * 8.0704536 * 31.839719 * -23.76924 *
*        1 *        3 * 80.704536 * 43.132717 * 37.571806 *
*        2 *        0 *         0 * 7.4278055 * -7.427805 *
*        2 *        1 *         0 * 12.739963 * -12.73996 *
*        2 *        2 * 8.0704536 * 41.566944 * -33.49625 *
*        2 *        3 * 80.704536 * 44.105440 * 36.598879 *
*        3 *        0 *         0 * 26.676696 * -26.67669 *
*        3 *        1 *         0 * 21.6

Type <CR> to continue or q to quit ==> 